In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [2]:
data=pd.read_csv('intern_data_ikarus.csv')
data.head()

,title,brand,description,price,categories,images,manufacturer,package_dimensions,country_of_origin,material,color,uniq_id
0,"GOYMFK 1pc Free Standing Shoe Rack, Multi-laye...",GOYMFK,"multiple shoes, coats, hats, and other items E...",$24.99,"['Home & Kitchen', 'Storage & Organization', '...",['https://m.media-amazon.com/images/I/416WaLx1...,GOYMFK,"2.36""D x 7.87""W x 21.6""H",China,Metal,White,02593e81-5c09-5069-8516-b0b29f439ded
1,"subrtex Leather ding Room, Dining Chairs Set o...",subrtex,subrtex Dining chairs Set of 2,NaN,"['Home & Kitchen', 'Furniture', 'Dining Room F...",['https://m.media-amazon.com/images/I/31SejUEW...,Subrtex Houseware INC,"18.5""D x 16""W x 35""H",NaN,Sponge,Black,5938d217-b8c5-5d3e-b1cf-e28e340f292e
2,Plant Repotting Mat MUYETOL Waterproof Transpl...,MUYETOL,NaN,$5.98,"['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...",['https://m.media-amazon.com/images/I/41RgefVq...,MUYETOL,"26.8""L x 26.8""W",NaN,Polyethylene,Green,b2ede786-3f51-5a45-9a5b-bcf856958cd8
3,"Pickleball Doormat, Welcome Doormat Absorbent ...",VEWETOL,The decorative doormat features a subtle textu...,$13.99,"['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...",['https://m.media-amazon.com/images/I/61vz1Igl...,Contrence,"24""L x 16""W",NaN,Rubber,A5589,8fd9377b-cfa6-5f10-835c-6b8eca2816b5
4,JOIN IRON Foldable TV Trays for Eating Set of ...,JOIN IRON Store,Set of Four Folding Trays With Matching Storag...,$89.99,"['Home & Kitchen', 'Furniture', 'Game & Recrea...",['https://m.media-amazon.com/images/I/41p4d4VJ...,NaN,"18.9""D x 14.2""W x 26""H",NaN,Iron,Grey Set of 4,bdc9aa30-9439-50dc-8e89-213ea211d66a


In [3]:
# === Step 1: Setup ===
!pip install -q pandas numpy sentence-transformers pinecone-client pillow tqdm scikit-learn joblib torch torchvision
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import joblib

print("✅ Environment ready")


✅ Environment ready


In [4]:
# === Step 2: Load Dataset ===
data = pd.read_csv('intern_data_ikarus.csv')
print("Shape:", data.shape)
data.head()


Shape: (312, 12)


,title,brand,description,price,categories,images,manufacturer,package_dimensions,country_of_origin,material,color,uniq_id
0,"GOYMFK 1pc Free Standing Shoe Rack, Multi-laye...",GOYMFK,"multiple shoes, coats, hats, and other items E...",$24.99,"['Home & Kitchen', 'Storage & Organization', '...",['https://m.media-amazon.com/images/I/416WaLx1...,GOYMFK,"2.36""D x 7.87""W x 21.6""H",China,Metal,White,02593e81-5c09-5069-8516-b0b29f439ded
1,"subrtex Leather ding Room, Dining Chairs Set o...",subrtex,subrtex Dining chairs Set of 2,NaN,"['Home & Kitchen', 'Furniture', 'Dining Room F...",['https://m.media-amazon.com/images/I/31SejUEW...,Subrtex Houseware INC,"18.5""D x 16""W x 35""H",NaN,Sponge,Black,5938d217-b8c5-5d3e-b1cf-e28e340f292e
2,Plant Repotting Mat MUYETOL Waterproof Transpl...,MUYETOL,NaN,$5.98,"['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...",['https://m.media-amazon.com/images/I/41RgefVq...,MUYETOL,"26.8""L x 26.8""W",NaN,Polyethylene,Green,b2ede786-3f51-5a45-9a5b-bcf856958cd8
3,"Pickleball Doormat, Welcome Doormat Absorbent ...",VEWETOL,The decorative doormat features a subtle textu...,$13.99,"['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...",['https://m.media-amazon.com/images/I/61vz1Igl...,Contrence,"24""L x 16""W",NaN,Rubber,A5589,8fd9377b-cfa6-5f10-835c-6b8eca2816b5
4,JOIN IRON Foldable TV Trays for Eating Set of ...,JOIN IRON Store,Set of Four Folding Trays With Matching Storag...,$89.99,"['Home & Kitchen', 'Furniture', 'Game & Recrea...",['https://m.media-amazon.com/images/I/41p4d4VJ...,NaN,"18.9""D x 14.2""W x 26""H",NaN,Iron,Grey Set of 4,bdc9aa30-9439-50dc-8e89-213ea211d66a


In [5]:
# === Step 3: Clean Text Columns ===
data.fillna("", inplace=True)

# Combine title + description + categories + material + color
data["text"] = (
    data["title"].astype(str) + " " +
    data["description"].astype(str) + " " +
    data["categories"].astype(str) + " " +
    data["material"].astype(str) + " " +
    data["color"].astype(str)
)

print("Sample combined text:")
print(data["text"].iloc[0][:500])


Sample combined text:
GOYMFK 1pc Free Standing Shoe Rack, Multi-layer Metal Shoe Cap Rack With 8 Double Hooks For Living Room, Bathroom, Hallway multiple shoes, coats, hats, and other items Easy to assemble: Includes all necessary hardware and instructions for easy assembly Versatile: Perfect for use in living rooms, bathrooms, hallways, and more ['Home & Kitchen', 'Storage & Organization', 'Clothing & Closet Storage', 'Shoe Organizers', 'Free Standing Shoe Racks'] Metal White


In [6]:
# === Step 4: Generate Text Embeddings ===
model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = []
for text in tqdm(data["text"], desc="Embedding products"):
    emb = model.encode(text)
    embeddings.append(emb)

embeddings = np.array(embeddings)
print("Embeddings shape:", embeddings.shape)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding products: 100%|██████████| 312/312 [00:31<00:00,  9.98it/s]

Embeddings shape: (312, 384)


In [7]:
# === Step 5: Save Embeddings Locally ===
emb_df = pd.DataFrame(embeddings)
emb_df["uniq_id"] = data["uniq_id"]
emb_df.to_csv("furniture_embeddings.csv", index=False)
joblib.dump(model, "recommender_model.joblib")

print("✅ Saved: furniture_embeddings.csv, recommender_model.joblib")


✅ Saved: furniture_embeddings.csv, recommender_model.joblib


In [8]:
# === Step 6: Quick Semantic Search ===
def recommend(query, top_n=5):
    query_emb = model.encode(query).reshape(1, -1)
    sims = cosine_similarity(query_emb, embeddings)[0]
    top_idx = sims.argsort()[-top_n:][::-1]
    return data.iloc[top_idx][["title", "price", "brand", "images"]]

recommend("wooden chair for dining", top_n=3)


,title,price,brand,images
30,"Flash Furniture Webb Commercial Grade 24"" Roun...",$140.00,Flash Furniture Store,['https://m.media-amazon.com/images/I/41IWb+KW...
72,Adeco Euro Style Fabric Arm Bench Chair Footst...,,Adeco Store,['https://m.media-amazon.com/images/I/41hUc8c+...
140,"Armen Living Julius 30"" Cream Faux Leather and...",,Armen Living Store,['https://m.media-amazon.com/images/I/31v34T0k...


In [9]:
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO

# === Step 7: Image Embeddings ===
resnet = models.resnet18(pretrained=True)
resnet.eval()
feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-1])

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def get_image_embedding(url):
    try:
        response = requests.get(eval(url)[0], timeout=5)
        image = Image.open(BytesIO(response.content)).convert('RGB')
        img_t = preprocess(image).unsqueeze(0)
        with torch.no_grad():
            emb = feature_extractor(img_t).squeeze().numpy()
        return emb
    except:
        return np.zeros((512,))

data["image_emb"] = data["images"].apply(get_image_embedding)
print("✅ Image embeddings generated")


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 55.7MB/s]


✅ Image embeddings generated


In [10]:
# === Step 8: Save Everything ===
data.to_csv("furniture_data_with_embeddings.csv", index=False)
joblib.dump(feature_extractor, "image_model.pt")

print("✅ All models and data saved.")


✅ All models and data saved.
